<a href="https://colab.research.google.com/github/sriharikrishna/siamcse23/blob/main/tapenade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tapenade mini benchmark

In this notebook we differentiate a small function with Tapenade in forward and reverse mode, then measure the run time of the obtained functions.

We use [Tapenade](http://tapenade.inria.fr:8080) and C for this. Apologies if the syntax highlighting for C is not perfect.

Download Tapenade and unpack it

In [ ]:
%%script bash
wget https://tapenade.gitlabpages.inria.fr/tapenade/distrib/tapenade_3.16.tar
tar -xvf tapenade_3.16.tar

tapenade_3.16/
tapenade_3.16/README.html
tapenade_3.16/bin/
tapenade_3.16/bin/tapenadeassert
tapenade_3.16/bin/linux/
tapenade_3.16/bin/linux/fortranParser
tapenade_3.16/bin/cppParserRun.sh
tapenade_3.16/bin/tapenadocker
tapenade_3.16/bin/tapenade.bat
tapenade_3.16/bin/tapenade
tapenade_3.16/bin/tapclass
tapenade_3.16/bin/fortranParserRun.sh
tapenade_3.16/bin/windows/
tapenade_3.16/bin/windows/fortranParser.exe
tapenade_3.16/bin/tapenadeaa
tapenade_3.16/bin/mac/
tapenade_3.16/ADFirstAidKit/
tapenade_3.16/ADFirstAidKit/adContext.c
tapenade_3.16/ADFirstAidKit/testMemSizec.c
tapenade_3.16/ADFirstAidKit/adOMP.h
tapenade_3.16/ADFirstAidKit/contextAD.sh
tapenade_3.16/ADFirstAidKit/adContextCPX.c
tapenade_3.16/ADFirstAidKit/admm_tapenade_interface.f90
tapenade_3.16/ADFirstAidKit/README.md
tapenade_3.16/ADFirstAidKit/OLDadBuffer.f
tapenade_3.16/ADFirstAidKit/OLDadStack.h
tapenade_3.16/ADFirstAidKit/validityTest.f
tapenade_3.16/ADFirstAidKit/PUSHPOPGeneralLib
tapenade_3.16/ADFirstAidKit/adDebug

--2023-02-13 22:02:33--  https://tapenade.gitlabpages.inria.fr/tapenade/distrib/tapenade_3.16.tar
Resolving tapenade.gitlabpages.inria.fr (tapenade.gitlabpages.inria.fr)... 128.93.193.23
Connecting to tapenade.gitlabpages.inria.fr (tapenade.gitlabpages.inria.fr)|128.93.193.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3034495 (2.9M) [application/x-tar]
Saving to: ‘tapenade_3.16.tar’

     0K .......... .......... .......... .......... ..........  1%  358K 8s
    50K .......... .......... .......... .......... ..........  3%  360K 8s
   100K .......... .......... .......... .......... ..........  5% 66.3M 5s
   150K .......... .......... .......... .......... ..........  6%  144M 4s
   200K .......... .......... .......... .......... ..........  8%  360K 5s
   250K .......... .......... .......... .......... .......... 10% 79.2M 4s
   300K .......... .......... .......... .......... .......... 11% 89.4M 3s
   350K .......... .......... .......... .........

## Input function

This is a simple function that takes an input vector of size `n` and produces an output vector of size `m`. It performs a somewhat arbitrary computation internally.

In [ ]:
%%writefile fun.c

void fun(int n, float a[n], int m, float b[m]) {
  float sum = 0.0;
  for(int i=0; i<n; i++) {
    sum += a[i]*a[i];
  }
  for(int i=0; i<m; i++) {
    b[i] = sum*i;
  }
}

Writing fun.c


In [ ]:
%%writefile rosenbrock.f
REAL*8 FUNCTION ROSENBROCK(x,n) RESULT(y)
INTEGER :: n
REAL*8 :: x(0:n)
y = SUM(100.d0*(x(1:n) - x(0:n-1)**2)**2
+ (1-x(0:n-1))**2)
END FUNCTION ROSENBROCK

Writing rosenbrock.f


In [ ]:
%%writefile rosenbrock.c
void rosenbrock(double const *control, double *out, int n) {
  int i;
  const double b[10];
  double result;
  for (i = 0; i < n; i += 2) {
    double c1 = (control[i + 1] - control[i] * control[i]);
    double  c2 = 1.0 - control[i];
    result += b * c1 * c1 + c2 * c2;
  }
  *out = result;
};

Overwriting rosenbrock.c


In [ ]:
%%bash
gcc rosenbrock.c

rosenbrock.c: In function ‘rosenbrock’:
rosenbrock.c:8:17: error: invalid operands to binary * (have ‘const double *’ and ‘double’)
    8 |     result += b * c1 * c1 + c2 * c2;
      |               ~ ^
      |               |
      |               const double *


CalledProcessError: ignored

In [ ]:
%%bash
/content/tapenade_3.16/bin/tapenade -d rosenbrock.f

Tapenade 3.16 (develop) -  9 Feb 2023 10:28 - Java 11.0.17 Linux
@@ TAPENADE_HOME=/content/tapenade_3.16/bin/..
File: /content/tapenade_3.16/bin/../bin/fortranParserRun.sh: line 19: tapenade: command not found
File: /content/tapenade_3.16/bin/../bin/fortranParserRun.sh: line 20: docker: command not found
File: /content/tapenade_3.16/bin/../bin/fortranParserRun.sh: line 21: docker: command not found
System: Parsing error in rosenbrock.f
Command: No root unit to differentiate
File: The code provided does not contain a top procedure
@@ Created ./rosenbrock_d.msg
job is completed (but doubts on parsing): total time 0.36 s


## Forward Mode AD

This is the result of applying Tapenade in forward mode to the function above, choosing `a` as independent input and `b` as dependent output. If you modify the function, you will have to manually run Tapenade (through the command line or web interface), as this step is not integrated in this notebook.

In [ ]:
%%bash
/content/tapenade_3.16/bin/tapenade -d fun.c

Tapenade 3.16 (develop) -  9 Feb 2023 10:28 - Java 11.0.17 Linux
@@ TAPENADE_HOME=/content/tapenade_3.16/bin/..
Tool: (DD99) Warning: Some features of C are not yet supported
Command: Took procedure fun as default differentiation root
Diff-liveness analysis turned off
@@ Created ./fun_d.c 
@@ Created ./fun_d.msg


In [ ]:
%%bash
cat fun_d.c

/*        Generated by TAPENADE     (INRIA, Ecuador team)
    Tapenade 3.16 (develop) -  9 Feb 2023 10:28
*/
/*
  Differentiation of fun in forward (tangent) mode:
   variations   of useful results: b[0:m-1]
   with respect to varying inputs: a[0:n-1] b[0:m-1]
   RW status of diff variables: a:(loc) a[0:n-1]:in b:(loc) b[0:m-1]:in-out
   Plus diff mem management of: a:in b:in
*/
void fun_d(int n, float a[n], float ad[n], int m, float b[m], float bd[m]) {
    float sum = 0.0;
    float sumd;
    sumd = 0.0;
    for (int i = 0; i < n; ++i) {
        sumd = sumd + 2*a[i]*ad[i];
        sum += a[i]*a[i];
    }
    for (int i = 0; i < m; ++i) {
        bd[i] = i*sumd;
        b[i] = sum*i;
    }
}


In [ ]:
%%bash
cat fun_d.msg

1 Tool: (DD99) Warning: Some features of C are not yet supported
2 Command: Took procedure fun as default differentiation root


## Reverse Mode AD

This is the result of applying Tapenade in reverse mode to the function above, again choosing `a` as independent input and `b` as dependent output. If you run Tapenade and it generates code that uses the AD run time library (for example, it calls `pushInteger4(...)` or similar functions), then you will need to obtain the ADFirstAidKit from the Tapenade website, which contains these functions. You may not be able to use this within this notebook.

In [ ]:
%%bash
/content/tapenade_3.16/bin/tapenade -b fun.c

Tapenade 3.16 (develop) -  9 Feb 2023 10:28 - Java 11.0.17 Linux
@@ TAPENADE_HOME=/content/tapenade_3.16/bin/..
Tool: (DD99) Warning: Some features of C are not yet supported
Command: Took procedure fun as default differentiation root
@@ Created ./fun_b.c 
@@ Created ./fun_b.msg


In [ ]:
%%bash
cat fun_b.c

/*        Generated by TAPENADE     (INRIA, Ecuador team)
    Tapenade 3.16 (develop) -  9 Feb 2023 10:28
*/
#include <adStack.h>

/*
  Differentiation of fun in reverse (adjoint) mode:
   gradient     of useful results: a[0:n-1] b[0:m-1]
   with respect to varying inputs: a[0:n-1] b[0:m-1]
   RW status of diff variables: a:(loc) a[0:n-1]:incr b:(loc)
                b[0:m-1]:in-out
   Plus diff mem management of: a:in b:in
*/
void fun_b(int n, float a[n], float ab[n], int m, float b[m], float bb[m]) {
    float sum = 0.0;
    float sumb = 0.0;
    sumb = 0.0;
    for (int i = m-1; i > -1; --i) {
        sumb = sumb + i*bb[i];
        bb[i] = 0.0;
    }
    for (int i = n-1; i > -1; --i)
        ab[i] = ab[i] + 2*a[i]*sumb;
}


In [ ]:
%%bash
cat fun_b.msg

1 Tool: (DD99) Warning: Some features of C are not yet supported
2 Command: Took procedure fun as default differentiation root


## Driver

This driver takes the problem sizes from the terminal input, sets up the input and output vectors, and calls the function and its derivatives while measuring the time taken. It assembles the full Jacobian matrix of the function by seeding with the full set of cartesian basis vectors. You probably do not need to modify the driver.

In [ ]:
%%writefile driver.c

#include <time.h>
#include <stdio.h>
#include <stdlib.h>

void fun(int n, float a[n], int m, float b[m]);
void fun_d(int n, float a[n], float ad[n], int m, float b[m], float bd[m]);
void fun_b(int n, float a[n], float ab[n], int m, float b[m], float bb[m]);

int main(int argc, char** argv) {
  /*
  Problem setup. Get input/output size, allocate memory, etc.
  */
  if(argc < 3) {
    printf("Usage: driver <num_inputs> <num_outputs>");
    return -1;
  }
  int n = atoi(argv[1]);
  int m = atoi(argv[2]);
  printf("===============================================\n");
  printf("Running function with %d inputs and %d outputs.\n",n,m);
  float* a = (float*)malloc(n*sizeof(float));
  float* a_seed = (float*)malloc(n*sizeof(float));
  for(int i=0; i<n; i++) {
    a[i] = (float)i;
    a_seed[i] = 0;
  }
  float* b = (float*)malloc(m*sizeof(float));
  float* b_seed = (float*)malloc(m*sizeof(float));
  for(int i=0; i<m; i++) {
    b_seed[i] = 0;
  }
  float* J_fwd = (float*)malloc(n*m*sizeof(float));
  float* J_rev = (float*)malloc(n*m*sizeof(float));
  clock_t tic, toc;
  double primaltime, difftime;

  /*
  Run the primal function and time it
  */
  tic = clock();
  fun(n, a, m, b);
  toc = clock() - tic;
  primaltime = (double)toc / (double)CLOCKS_PER_SEC;
  printf("primal time: %f\n", primaltime);

  printf("\nComputing one row/column of Jacobian:\n");
  /*
  Run the forward-mode once
  */
  tic = clock();
  a_seed[0] = 1.0;
  fun_d(n, a, a_seed, m, b, &J_fwd[0]);
  a_seed[0] = 0.0;
  toc = clock() - tic;
  difftime = (double)toc / (double)CLOCKS_PER_SEC;
  printf("forward Jv time: %f (factor: %f X)\n", difftime, difftime/primaltime);

  /*
  Run the reverse-mode once
  */
  tic = clock();
  b_seed[0] = 1.0;
  fun_b(n, a, &J_rev[0], m, b, b_seed);
  b_seed[0] = 0.0;
  toc = clock() - tic;
  difftime = (double)toc / (double)CLOCKS_PER_SEC;
  printf("reverse vJ time: %f (factor: %f X)\n", difftime, difftime/primaltime);

  printf("\nComputing the entire Jacobian:\n");
  /*
  Run the forward-mode n times (enough to obtain the entire Jacobian).
  */
  tic = clock();
  for(int i=0; i<n; i++) {
    a_seed[i] = 1.0;
    fun_d(n, a, a_seed, m, b, &J_fwd[i*m]);
    a_seed[i] = 0.0;
  }
  toc = clock() - tic;
  difftime = (double)toc / (double)CLOCKS_PER_SEC;
  printf("forward J time: %f (factor: %f X)\n", difftime, difftime/primaltime);

  /*
  Run the reverse-mode m times (enough to obtain the entire Jacobian).
  */
  tic = clock();
  for(int i=0; i<m; i++) {
    b_seed[i] = 1.0;
    fun_b(n, a, &J_rev[i*n], m, b, b_seed);
    b_seed[i] = 0.0;
  }
  toc = clock() - tic;
  difftime = (double)toc / (double)CLOCKS_PER_SEC;
  printf("reverse J time: %f (factor: %f X)\n", difftime, difftime/primaltime);

  /*
  Compare the computed Jacobians
  */
  for(int i=0; i<n; i++) {
    for(int j=0; j<m; j++) {
      if(J_fwd[i*m+j] != J_rev[j*n+i]) {
        printf("Jacobian from fwd and rev does not match!: %f != %f\n", J_fwd[i*m+j], J_rev[j*n+i]);
        return -1;
      }
    }
  }
  printf("Jacobian from fwd and rev matches!\n\n");

  free(a); free(b); free(a_seed); free(J_fwd); free(J_rev); free(b_seed);
  return 0;
}

## Compile

Compile our benchmark.

In [ ]:
%%script bash

# if the AD runtime library is not yet loaded, download and compile it now.
if [ ! -f ADFirstAidKit/adBuffer.h ]; then
    wget tapenade.inria.fr:8080/tapenade/ADFirstAidKit.tar
    tar vxf ADFirstAidKit.tar
    (cd ADFirstAidKit && gcc -c -O3 -w adBuffer.c adStack.c adContext.c && ar -crs libAD.a *.o)
fi

# finally, install 
gcc -O3 -I./ADFirstAidKit -L./ADFirstAidKit fun.c fun_d.c fun_b.c driver.c -lAD -o perftest

In [ ]:
%%script bash
export TAPENADE_HOME=${PWD}/tapenade_3.16
export PATH=${TAPENADE_HOME}/bin:${PATH}
echo $PATH
tapenade -v

/content/tapenade_3.16/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
Tapenade 3.16 (develop) -  9 Feb 2023 10:28 - Java 11.0.17 Linux
@@ TAPENADE_HOME=/content/tapenade_3.16/bin/..
Command: File not found: -v
Command: No input files
tapenade: Differentiation failed


CalledProcessError: ignored

In [ ]:
%%script bash
/content/tapenade_3.16/bin/tapenade -v

Tapenade 3.16 (develop) -  9 Feb 2023 10:28 - Java 11.0.17 Linux
@@ TAPENADE_HOME=/content/tapenade_3.16/bin/..
Command: File not found: -v
Command: No input files
tapenade: Differentiation failed


CalledProcessError: ignored

## Run

Here we run the benchmark with just two settings: 1 input and 10000 outputs, and 10000 inputs and 1 output. This should highlight the differences between forward and reverse mode well.

In [ ]:
%%script bash

./perftest 1 10000

In [ ]:
%%script bash

./perftest 10000 1

## Running the benchmark for a range of input and output sizes

No need to understand this in detail. We loop over some input and output sizes, and filter the output of perftest to get only the overhead factors. We store all of them in `timings.txt`, which we will plot in the next field.

In [ ]:
%%script bash
echo "" > timings.txt
for N_IN in 1 1000 2000 3000 4000 5000
do
  for N_OUT in 1 1000 2000 3000 4000 5000
  do
    OUTPUT=$(./perftest $N_IN $N_OUT)
    TIME_FWD=$(grep "forward J time" <<< "$OUTPUT" | awk '{print $6}')
    TIME_REV=$(grep "reverse J time" <<< "$OUTPUT" | awk '{print $6}')
    echo "$N_IN $N_OUT $TIME_FWD $TIME_REV" >> timings.txt
  done
done

wc -l timings.txt

## Plotting the time overhead factors
We use Python for this. Read `timings.txt` and generate a plot.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
time_data = np.genfromtxt('timings.txt', delimiter=' ')

n_vec = time_data[:,0]
m_vec = time_data[:,1]
fwd_time = time_data[:,2]
rev_time = time_data[:,3]

fig, (ax0,ax1) = plt.subplots(subplot_kw={"projection": "3d"}, nrows=1, ncols=2, figsize=(12,8), dpi= 100)
for ax in (ax0,ax1):
  ax.set_xlabel('#inputs')
  ax.set_ylabel('#outputs')
  ax.invert_yaxis()
ax0.plot_trisurf(n_vec,m_vec,fwd_time)
ax1.plot_trisurf(n_vec,m_vec,rev_time)
ax0.title.set_text('forward mode overhead')
ax1.title.set_text('reverse mode overhead')

def plot_fun(ax, n_vals, m_vals, times, maxval, title=None):
  # Plot the surface.
  X, Y = np.meshgrid(list(n_vals.keys()), list(m_vals.keys()))
  surf = ax.plot_surface(X, Y, times, cmap=cm.coolwarm,
                        antialiased=True)
  # Customize the z  axis.
  ax.set_zlim(0, maxval*1.1)

  coordsn = list(n_vals.keys())
  ax.set_xticks(coordsn, ["$4^{:d}$".format(i) for i in coordsn])
  
  coordsm = list(m_vals.keys())
  ax.set_yticks(coordsm, ["$4^{:d}$".format(i) for i in coordsm])
  
  ax.invert_yaxis()
  
  ax.title.set_text(title)
  ax.set_xlabel('#inputs')
  ax.set_ylabel('#outputs')

# Exercises

Here are some ideas to try out:


1.   Change the number of inputs and outputs and observe how the run times and overhead factors change for forward and reverse mode.
2.   Re-run the same setting a few times. The notebook run times are a little noisy, probably you will get slightly different numbers each time.
2.   Change the function `fun` to something you actually want to compute (make sure the function header stays the same if you want to leave the rest of the notebook as is). You will need to use Tapenade to obtain the derivative functions `fun_d` and `fun_b` yourself, as the notebook does not automate this step.

